In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/Users/hintont/Dev/packages")

import warnings
warnings.simplefilter("ignore")


In [ ]:
import os
import matplotlib.pyplot as plt

import csi.insar as ir
import csi.TriangularPatches as triangleFault
import csi.imagedownsampling as imdown
import csi.imagecovariance as imcov

In [ ]:
name = "ridgecrest"
main = "/Users/hintont/Dev/projects/Ridgecrest"
confdir = os.path.join(main,'config/')
gfsdir = os.path.join(main,'config/gfs/')                      
datadir = os.path.join(main, "data/insar/")
resdir = os.path.join(main,'results/')
figdir = os.path.join(main,'fig/')
faultdir = os.path.join(main,'data/fault/')
lon0= 360-117.5
lat0= 35.7

MINLON, MAXLON, MINLAT, MAXLAT = 360-118.1, 360-117.0 ,35.3, 36.2

In [ ]:
faults = []
for igeom in [1,3]:
    fault = triangleFault(f"Fault {igeom}", lon0=lon0, lat0=lat0, utmzone="11")
    fault.readPatchesFromFile(os.path.join(faultdir,f'fault{igeom}.basemesh.triangles'), donotreadslip=True, readpatchindex=False)
    fault.setTrace(delta_depth=0.2)
    fault.discretize(every=0.2, fracstep=0.05)
    fault.xf = fault.xi
    fault.yf = fault.yi
    faults.append(fault)

In [ ]:
subfolder = 'fielding/S1AB/A064/int_0704-0710/'
sarname = 'S1_A064_20190704-0710_v3_unw_m_3asec'
e = 'S1_A064_20190704-0710_v3_east_3asec'
n ='S1_A064_20190704-0710_v3_north_3asec'
u = 'S1_A064_20190704-0710_v3_up_3asec'

sa = ir(sarname, utmzone="11", lon0=lon0, lat0=lat0)
sa.read_from_grd( datadir+subfolder+sarname+'.grd', 
                 los=[ datadir+subfolder+e+'.grd', 
                      datadir+subfolder+n+'.grd', 
                      datadir+subfolder+u+'.grd' ], factor=1.)

# Crop to a given area
sa.select_pixels(MINLON, MAXLON, MINLAT, MAXLAT)
sa.checkNaNs()

In [ ]:
%matplotlib inline

sa.getprofile("a profile", lon0, lat0, 50., 45., 1.)
sa.plotprofile("a profile", fault=faults[0])

In [ ]:
%matplotlib inline

startWindowSize, minimumWindowSize, chardist, expodist, tol = 10., 1.25, 1.5, 0.7, 0.005
threshold, smooth, damping = 0.0035, 7., 0.005

downsampler = imdown('downsampling {}'.format(sa.name), sa, faults)
downsampler.initialstate(startWindowSize, minimumWindowSize, 
                         tolerance=tol, plot=False)
# downsampler.dataBased(threshold, plot=False, quantity='curvature', smooth=smooth)
downsampler.distanceBased(chardist=chardist, expodist=expodist, plot=True)
downsampler.reject_pixels_fault(0.5, faults)
downsampler.writeDownsampled2File(datadir+'{}'.format(sa.name), rsp=True) 

In [ ]:
#Create a covariance estimation object
covar1 = imcov(sarname+' covariance', sa, verbose=True)
covar1.selectedZones = []
covar1.maskOut([ 360-117.88,360-117.3,35.53,35.9])
#covar1.computeCovariance(function='gauss', frac=0.005, every=1.0, distmax=40.,tol=1e-10)
covar1.computeCovariance(function='exp', frac=0.005, every=0.5, distmax=35.,tol=1e-10)
# Plot
covar1.plot(data='all', plotData=True,savefig=True,savedir=datadir)
# Write to a file
covar1.write2file(savedir=datadir)

In [ ]:
%matplotlib widget
sa = ir(sarname, utmzone="11", lon0=lon0, lat0=lat0)
sa.read_from_varres( datadir+'{}'.format(sarname))
sa.buildCd(0.027436557793776566, 32.21656668629711, function='exp')
sa.plot()

In [ ]:
datasets = [sa]
for fault in faults:
    fault.initializeslip()
    for data in datasets:
        fault.buildGFs(data, slipdir="sd")
    fault.assembleGFs(datasets, slipdir="sd", polys=None)
    fault.saveGFs(outputDir=(resdir + "greens"))

In [ ]:
for fault in faults:
    fault.initializeslip()
    for data in datasets:
        sname = os.path.join(resdir+"greens",'{}_{}_SS.gf'.format(fault.name.replace(' ','_'), data.name.replace(' ','_')))
        dname = os.path.join(resdir+"greens",'{}_{}_DS.gf'.format(fault.name.replace(' ','_'), data.name.replace(' ','_')))
        fault.setGFsFromFile(data, strikeslip=sname, dipslip=dname, vertical=True)
    
    fault.assembleGFs(datasets, slipdir='sd')
    fault.assembleCd(datasets)
    fault.assembled(datasets)

In [ ]:
import csi.multifaultsolve as multiflt
import csi.transformation as transformation
import numpy as np

trans = transformation('Orbits and reference frame', utmzone="11", lon0=lon0, lat0=lat0)
trans.buildGFs(datasets, [3]*len(datasets))
trans.assembleGFs(datasets)
trans.assembled(datasets)
trans.assembleCd(datasets)

slv = multiflt(name, faults+[trans])
slv.assembleGFs()
for fault in faults:
   fault.buildCm(1., 1.)
trans.buildCm(1000.)
slv.assembleCm()



#slv.Cd /= 10.
#slv.G *= 100.
bounds = []
for f in range(len(faults)):
   for i in range(faults[f].N_slip):
       bounds.append([0,10.])
   for i in range(faults[f].N_slip):
       bounds.append([-3., 3.])   
for i in range(trans.TransformationParameters):
   bounds.append([-100., 100.])
mprior = np.zeros((len(bounds),))

slv.ConstrainedLeastSquareSoln(bounds=bounds, 
                              iterations=10,
                              method='L-BFGS-B',
                              mprior=mprior, 
                              tolerance=1e-4, 
                              maxfun=100, 
                              checkIter=True)
# slv.distributem()
# trans.removePredictions(datasets)

In [ ]:
slv.distributem()
trans.removePredictions(datasets)

In [ ]:
import pickle
with open(datadir + sarname + "_mpost.pickle", 'wb') as f:
    pickle.dump(slv.mpost, f)

In [ ]:
import csi.geodeticplot as geoplt

# Plot predictions and inferred slip
for data in datasets:
   data.buildsynth(slv.faults)

for f in range(len(faults)):
   gp = geoplt(figure=f, lonmin=-118., lonmax=-116.5, latmin=35., latmax=36.4,
           figsize=[(10,5),(10,5)])
   #gp.drawCoastlines(drawLand=True, parallels=0.2, meridians=0.2, drawOnFault=True)
   #gp.faulttrace(faults[0], color='r')
   gp.faultTents(faults[f], colorbar=True, slip='strikeslip', plot_on_2d=False,
                            revmap=False, method='scatter')
   #gp.faultTents(faults[1], colorbar=True, slip='strikeslip', plot_on_2d=False,
   #                         revmap=False, method='scatter')
   #gp.gps(gpsData[0], data=['data', 'synth'], color=['r', 'b'], scale=1., legendscale=0.1)
#    gp = geoplt(figure=2*f, lonmin=-118., lonmax=-116.5, latmin=35., latmax=36.4,
#            figsize=[(10,5),(10,5)])
#    #gp.drawCoastlines(drawLand=True, parallels=0.2, meridians=0.2, drawOnFault=True)
#    #gp.faulttrace(faults[0], color='r')
#    #gp.faultTents(faults[0], colorbar=True, slip='dipslip', plot_on_2d=False,
#    #                         revmap=False, method='scatter')
#    gp.faultTents(faults[1], colorbar=True, slip='strikeslip', plot_on_2d=False,
#                             revmap=False, method='scatter')
for sar in datasets:
   sar.plot(faults=faults, data='data', norm=[-0.5, 0.5], show=False)
   sar.plot(faults=faults, data='synth', norm=[-0.5, 0.5], show=False)
   sar.plot(faults=faults, data='res', norm=[-0.15, 0.15], show=False)